# SQL Alchemy 

Enable to interact with SQL on the Python script

In [1]:
#!pip install SQLAlchemy

## Connect to DB: Create Engine

In [44]:
import sqlalchemy as db 

In [45]:
DATABASE_URL = 'sqlite:///sql/chinook.db' ## link the the .db
engine = db.create_engine(DATABASE_URL) ## connect to engine
print(engine)

Engine(sqlite:///sql/chinook.db)


In [47]:
db.inspect(engine).get_table_names() ## show all table

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'sqlite_sequence',
 'sqlite_stat1',
 'tracks']

In [48]:
db.inspect(engine).get_columns('albums')


[{'name': 'AlbumId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'Title',
  'type': NVARCHAR(length=160),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'ArtistId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

In [50]:
col = []
for c in db.inspect(engine).get_columns('albums'):
    col.append(c['name'])
col 

['AlbumId', 'Title', 'ArtistId']

## Connect to DB: Create Connection

In [51]:
connection = engine.connect()

In [52]:
metadata = db.MetaData()
type(metadata)

sqlalchemy.sql.schema.MetaData

In [53]:
metadata.tables.keys() ## it's empty till we create the table objects

dict_keys([])

In [54]:
metadata.sorted_tables

[]

## Create Table Objects

In [55]:
db.inspect(engine).get_table_names()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'sqlite_sequence',
 'sqlite_stat1',
 'tracks']

In [56]:
albums = db.Table('albums', metadata, autoload=True, autoload_with=engine)
artists = db.Table('artists', metadata, autoload=True, autoload_with=engine)
tracks = db.Table('tracks', metadata, autoload=True, autoload_with=engine)
genres = db.Table('genres', metadata, autoload=True, autoload_with=engine)

In [57]:
type(albums)

sqlalchemy.sql.schema.Table

In [59]:
metadata.tables.keys()

dict_keys(['albums', 'artists', 'tracks', 'genres', 'media_types'])

In [61]:
metadata.sorted_tables[0]

Table('artists', MetaData(), Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False), Column('Name', NVARCHAR(length=120), table=<artists>), schema=None)

# Query in SQLAlchemy

In [65]:
# Equivalent to 'SELECT * FROM albums'
stmt = db.select([albums])
result_proxy = connection.execute(stmt) ## return the object from class ResultProxy

In [68]:
results = result_proxy.fetchall() ## Fetch the result into a list
results[:5]

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3)]

In [69]:
print(len(results))

347


### select()

In [23]:
## select()
print(tracks.columns.keys())


['TrackId', 'Name', 'AlbumId', 'MediaTypeId', 'GenreId', 'Composer', 'Milliseconds', 'Bytes', 'UnitPrice']


In [24]:
print(db.select([tracks.columns.Name, tracks.columns.TrackId])) ## return the syntax

SELECT tracks."Name", tracks."TrackId" 
FROM tracks


In [26]:
s = db.select([tracks.columns.TrackId, tracks.columns.Name, tracks.c.Composer])
connection.execute(s).fetchmany(5)

[(1, 'For Those About To Rock (We Salute You)', 'Angus Young, Malcolm Young, Brian Johnson'),
 (2, 'Balls to the Wall', None),
 (3, 'Fast As a Shark', 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 (4, 'Restless and Wild', 'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 (5, 'Princess of the Dawn', 'Deaffy & R.A. Smith-Diesel')]

### where()

In [29]:
s = db.select([tracks.c.TrackId, tracks.c.Name, tracks.c.GenreId]).where(tracks.c.GenreId == 5)
print(s)

SELECT tracks."TrackId", tracks."Name", tracks."GenreId" 
FROM tracks 
WHERE tracks."GenreId" = :GenreId_1


In [30]:
connection.execute(s).fetchmany(5)

[(111, 'Money', 5),
 (112, 'Long Tall Sally', 5),
 (113, 'Bad Boy', 5),
 (114, 'Twist And Shout', 5),
 (115, 'Please Mr. Postman', 5)]

### group_by()

In [33]:
s = db.select([tracks.c.GenreId, db.func.count(tracks)]).group_by(tracks.c.GenreId)
print(s)

SELECT tracks."GenreId", count(tracks."TrackId") AS count_1 
FROM tracks GROUP BY tracks."GenreId"


In [35]:
connection.execute(s).fetchmany(5)

[(1, 1297), (2, 130), (3, 374), (4, 332), (5, 12)]

### join()

In [36]:
s = db.select(
        [tracks.c.TrackId, tracks.c.Name, genres.c.Name]
    ).select_from(tracks.join(genres)).where(genres.c.Name == "Jazz")
print(s)

SELECT tracks."TrackId", tracks."Name", genres."Name" AS "Name_1" 
FROM tracks JOIN genres ON genres."GenreId" = tracks."GenreId" 
WHERE genres."Name" = :Name_2


In [37]:
connection.execute(s).fetchmany(5)

[(63, 'Desafinado', 'Jazz'),
 (64, 'Garota De Ipanema', 'Jazz'),
 (65, 'Samba De Uma Nota Só (One Note Samba)', 'Jazz'),
 (66, 'Por Causa De Você', 'Jazz'),
 (67, 'Ligia', 'Jazz')]

## Use Textual SQL

In [70]:
SQL_script = '''
SELECT
    t1.TrackId, 
    t1.name AS Track_Name,
    t2.name AS Genres_Name
FROM tracks AS t1
JOIN genres AS t2 
ON t1.GenreId = t2.GenreId
WHERE t2.Name == 'Jazz'
LIMIT 3
'''

s = db.sql.text(SQL_script)

In [73]:
connection.execute(s).fetchmany(5)

[(63, 'Desafinado', 'Jazz'),
 (64, 'Garota De Ipanema', 'Jazz'),
 (65, 'Samba De Uma Nota Só (One Note Samba)', 'Jazz')]

In [75]:
print(s)


SELECT
    t1.TrackId, 
    t1.name AS Track_Name,
    t2.name AS Genres_Name
FROM tracks AS t1
JOIN genres AS t2 
ON t1.GenreId = t2.GenreId
WHERE t2.Name == 'Jazz'
LIMIT 3



In [76]:
## Pandas for much better output
import pandas as pd 
df = pd.read_sql(s, connection)
df

,TrackId,Track_Name,Genres_Name
0,63,Desafinado,Jazz
1,64,Garota De Ipanema,Jazz
2,65,Samba De Uma Nota Só (One Note Samba),Jazz
